In [12]:
#Number of vehicles
numVehicles = 5
#auto 1: 
#capacity = 39
#costs per km = 1

#auto 2: 
#capacity = 39
#costs per km = 1

#set of cities
cities = {'Ams', 'Rot', 'Hag', 'Utr', 'Ein', 'Til', 'Gro'}

## start and end
start = 'Ams'

## Capacity of each vehicle
Q = {1: 39, 2:39, 3:16, 4: 16, 5:16}

## cost per km
cKm = {1:1, 2:1, 3:0.5, 4:0.5, 5:0.5}

# Demand at each location
q = {'Ams':0, 'Rot':14, 'Hag':3, 'Utr':6, 'Ein':16, 'Til':15, 'Gro':5}

## Distances between cities
d = {('Ams', 'Ams'): 0,
 ('Ams', 'Rot'): 148,
 ('Ams', 'Hag'): 55,
 ('Ams', 'Utr'): 32,
 ('Ams', 'Ein'): 70,
 ('Ams', 'Til'): 140,
 ('Ams', 'Gro'): 73,
 ('Rot', 'Ams'): 148,
 ('Rot', 'Rot'): 0,
 ('Rot', 'Hag'): 93,
 ('Rot', 'Utr'): 180,
 ('Rot', 'Ein'): 99,
 ('Rot', 'Til'): 12,
 ('Rot', 'Gro'): 72,
 ('Hag', 'Ams'): 55,
 ('Hag', 'Rot'): 93,
 ('Hag', 'Hag'): 0,
 ('Hag', 'Utr'): 85,
 ('Hag', 'Ein'): 20,
 ('Hag', 'Til'): 83,
 ('Hag', 'Gro'): 28,
 ('Utr', 'Ams'): 32,
 ('Utr', 'Rot'): 180,
 ('Utr', 'Hag'): 85,
 ('Utr', 'Utr'): 0,
 ('Utr', 'Ein'): 100,
 ('Utr', 'Til'): 174,
 ('Utr', 'Gro'): 99,
 ('Ein', 'Ams'): 70,
 ('Ein', 'Rot'): 77,
 ('Ein', 'Hag'): 20,
 ('Ein', 'Utr'): 100,
 ('Ein', 'Ein'): 0,
 ('Ein', 'Til'): 85,
 ('Ein', 'Gro'): 49,
 ('Til', 'Ams'): 140,
 ('Til', 'Rot'): 12,
 ('Til', 'Hag'): 83,
 ('Til', 'Utr'): 174,
 ('Til', 'Ein'): 85,
 ('Til', 'Til'): 0,
 ('Til', 'Gro'): 73,
 ('Gro', 'Ams'): 73,
 ('Gro', 'Rot'): 72,
 ('Gro', 'Hag'): 28,
 ('Gro', 'Utr'): 99,
 ('Gro', 'Ein'): 49,
 ('Gro', 'Til'): 73,
 ('Gro', 'Gro'): 0}

In [13]:
import pyomo.environ as pe
import pyomo.opt as po

In [14]:
model = pe.ConcreteModel()

In [15]:
model.N = pe.Set(initialize =cities ,ordered = False)
model.K = pe.RangeSet(1, numVehicles)

In [16]:
model.x = pe.Var(model.N, model.N,model.K, domain = pe.Binary)
model.u = pe.Var(model.N, model.K ,domain = pe.NonNegativeReals)

In [17]:
objExpr = sum(model.x[i,j,k] * d[i,j] * cKm[k] for i in model.N for j in model.N for k in model.K)
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [18]:
# Tour leaves each city
model.cnstrOut = pe.ConstraintList()
for j in model.N:
    if j != start:
        expression = sum(model.x[i,j,k] for i in model.N for k in model.K) == 1
        model.cnstrOut.add(expression)
    
# Flow balance for each vehicle
model.flowBalance = pe.ConstraintList()
for j in model.N: 
    for k in model.K:
        expression = sum(model.x[i,j,k] for i in model.N) == sum(model.x[j,h,k] for h in model.N)
        model.flowBalance.add(expression)

## Each vehicle can be used at most once
model.vehicleUsage = pe.ConstraintList()
for k in model.K:
    expression = sum(model.x[start,j,k] for j in model.N) <= 1
    model.vehicleUsage.add(expression)
        
# Subtours elimination constraints
model.cnstrSubTour = pe.ConstraintList()
for i in model.N:
    for j in model.N:
        if i != j and j != start:
            for k in model.K:
                expression = model.u[i,k] + q[j] <= model.u[j,k] + max(Q[k],q[j]) *(1 - model.x[i,j,k])
                model.cnstrSubTour.add(expression)

# Max Capacity
model.cnstrCapl = pe.ConstraintList()
for i in model.N:
    for k in model.K:
        expression = model.u[i,k] <= Q[k]
        model.cnstrCapl.add(expression)

model.cnstrCap2 = pe.ConstraintList()
for i in model.N:
    for k in model.K:
        expression = model.u[i,k] >= q[i] 
        model.cnstrCap2.add(expression)            
            
            
## Starting point
model.cnstrStrat = pe.ConstraintList()
for k in model.K:
    expression = model.u[start,k] == 0
    model.cnstrStrat.add(expression)



## Prevent loops on a single node
model.cnstrSingleLoop = pe.ConstraintList()
for i in model.N:
    for k in model.K:
        expression = model.x[i,i,k] == 0
        model.cnstrSingleLoop.add(expression)
    


In [19]:
#model.pprint()
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-14
Read LP format model from file C:\Users\timva\AppData\Local\Temp\tmppdltpn1k.pyomo.lp
Reading time = 0.01 seconds
x281: 337 rows, 281 columns, 1316 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 337 rows, 281 columns and 1316 nonzeros
Model fingerprint: 0x3fcb3c06
Variable types: 36 continuous, 245 integer (245 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 648.0000000
Presolve removed 222 rows and 131 columns
Presolve time: 0.00s
Presolved: 115 rows, 150 columns, 870 nonzeros
Variable types: 21 continuous, 129 integer (129 binary)

Root relaxation: objective 1.390000e+02, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes 

In [20]:
print(result.solver.status)
print(result.solver.termination_condition)

ok
optimal


In [21]:
print("Objective value = "+ str(pe.value(model.obj)))

Objective value = 397.0


In [22]:
print("Tour:\n")
for k in model.K:
    for i in model.N:
        for j in model.N:
            if(pe.value(model.x[i,j,k]) > 0):
                print(str(model.x[i,j,k]) + " = " + str(pe.value(model.x[i,j,k])))

print("\nAccumulated demand:\n")
for i in model.N:
    for k in model.K:
        if(pe.value(model.u[i,k]) > 0):
            print(str(model.u[i,k]) + " = " + str(pe.value(model.u[i,k])))
            


Tour:

x[Rot,Gro,2] = 1.0
x[Hag,Til,2] = 1.0
x[Gro,Ams,2] = 1.0
x[Ams,Hag,2] = 1.0
x[Til,Rot,2] = 1.0
x[Ams,Ein,3] = 1.0
x[Ein,Ams,3] = 1.0
x[Utr,Ams,5] = 1.0
x[Ams,Utr,5] = 1.0

Accumulated demand:

u[Rot,1] = 39.0
u[Rot,2] = 34.0
u[Rot,3] = 14.0
u[Rot,4] = 14.0
u[Rot,5] = 14.0
u[Hag,1] = 3.0
u[Hag,2] = 5.0
u[Hag,3] = 3.0
u[Hag,4] = 3.0
u[Hag,5] = 3.0
u[Gro,1] = 39.0
u[Gro,2] = 39.0
u[Gro,3] = 5.0
u[Gro,4] = 5.0
u[Gro,5] = 5.0
u[Utr,1] = 6.0
u[Utr,2] = 6.0
u[Utr,3] = 6.0
u[Utr,4] = 6.0
u[Utr,5] = 6.0
u[Ein,1] = 39.0
u[Ein,2] = 39.0
u[Ein,3] = 16.0
u[Ein,4] = 16.0
u[Ein,5] = 16.0
u[Til,1] = 15.0
u[Til,2] = 20.0
u[Til,3] = 15.0
u[Til,4] = 15.0
u[Til,5] = 15.0
